In [6]:
!conda install -c conda-forge pandas pandas-profiling matplotlib openpyxl -y

Solving environment: ...working... done

# All requested packages already installed.



In [7]:
pip install pycaret

Note: you may need to restart the kernel to use updated packages.


In [9]:
from pycaret.classification import *

In [10]:
import pandas as pd
df = pd.read_csv('C:/Users/mleop/OneDrive/Desktop/MSDS600/prepped_churn_data_final2.csv' , index_col='customerID')
df

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,monthly_tenure_ratio
customerID,,,,,,,,
7590-VHVEG,1,0,0,0,29.85,29.85,0,29.850000
5575-GNVDE,34,1,1,1,56.95,1889.50,0,1.675000
3668-QPYBK,2,1,0,1,53.85,108.15,1,26.925000
7795-CFOCW,45,0,1,2,42.30,1840.75,0,0.940000
9237-HQITU,2,1,0,0,70.70,151.65,1,35.350000
...,...,...,...,...,...,...,...,...
6840-RESVB,24,1,1,1,84.80,1990.50,0,3.533333
2234-XADUH,72,1,1,3,103.20,7362.90,0,1.433333
4801-JZAZL,11,0,0,0,29.60,346.45,0,2.690909


In [11]:
automl = setup(df, target='Churn')

,Description,Value
0,session_id,3186
1,Target,Churn
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(7043, 8)"
5,Missing Values,False
6,Numeric Features,4
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [14]:
automl[3]

{'parameter': 'Hyperparameters',
 'auc': 'AUC',
 'confusion_matrix': 'Confusion Matrix',
 'threshold': 'Threshold',
 'pr': 'Precision Recall',
 'error': 'Prediction Error',
 'class_report': 'Class Report',
 'rfe': 'Feature Selection',
 'learning': 'Learning Curve',
 'manifold': 'Manifold Learning',
 'calibration': 'Calibration Curve',
 'vc': 'Validation Curve',
 'dimension': 'Dimensions',
 'feature': 'Feature Importance',
 'feature_all': 'Feature Importance (All)',
 'boundary': 'Decision Boundary',
 'lift': 'Lift Chart',
 'gain': 'Gain Chart',
 'tree': 'Decision Tree',
 'ks': 'KS Statistic Plot'}

In [15]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.8010,0.8378,0.4820,0.6755,0.5622,0.4381,0.4488,0.0060
lr,Logistic Regression,0.7998,0.8412,0.4950,0.6656,0.5672,0.4407,0.4492,0.6780
ada,Ada Boost Classifier,0.7976,0.8373,0.5172,0.6508,0.5752,0.4448,0.4505,0.0410
gbc,Gradient Boosting Classifier,0.7953,0.8395,0.5011,0.6485,0.5648,0.4339,0.4404,0.0970
ridge,Ridge Classifier,0.7947,0.0000,0.4177,0.6881,0.5191,0.3983,0.4188,0.0050
lightgbm,Light Gradient Boosting Machine,0.7890,0.8237,0.5149,0.6258,0.5643,0.4269,0.4309,0.0230
rf,Random Forest Classifier,0.7736,0.7966,0.4896,0.5893,0.5337,0.3862,0.3897,0.1130
knn,K Neighbors Classifier,0.7684,0.7430,0.4498,0.5831,0.5072,0.3591,0.3647,0.4110
svm,SVM - Linear Kernel,0.7653,0.0000,0.3796,0.5985,0.4393,0.3100,0.3321,0.0110
et,Extra Trees Classifier,0.7574,0.7725,0.4920,0.5481,0.5177,0.3565,0.3579,0.0940


In [16]:
best_model

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

In [17]:
df.iloc[-2:-1].shape

(1, 8)

In [19]:
predict_model(best_model, df.iloc[-2:-1])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Linear Discriminant Analysis,0.0000,0,0.0000,0.0000,0.0000,0.0000,0.0000


,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,monthly_tenure_ratio,Label,Score
customerID,,,,,,,,,,
8361-LTMKD,4,1,0,1,74.4,306.6,1,18.6,0,0.5242


In [20]:
save_model(best_model, 'LDA')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='Churn',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strate...
                 ('dummy', Dummify(target='Churn')),
                 ('fix_perfect', Remove_100(target='Churn')),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs

In [27]:
import pickle

with open('LDA_model.pk', 'wb') as f:
    pickle.dump(best_model, f)

In [30]:
with open('LDA_model.pk', 'rb') as f:
    loaded_model = pickle.load(f)

In [32]:
loaded_lda = load_model('LDA')

Transformation Pipeline and Model Successfully Loaded


In [33]:
predict_model(loaded_lda, new_data)

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,monthly_tenure_ratio,Label,Score
customerID,,,,,,,,,
8361-LTMKD,4,1,0,1,74.4,306.6,18.6,0,0.5242


In [40]:
from IPython.display import Code

Code('predict_churn.py')

import pandas as pd
from pycaret.classification import predict_model, load_model

def load_data(filepath):
    df = pd.read_csv(filepath, index_col='customerID')
    return df


def make_predictions(df):
    model = load_model('LDA')
    predictions = predict_model(model, data=df)
    predictions.rename({'Label': 'Churn_prediction'}, axis=1, inplace=True)
    predictions['Churn_prediction'].replace({1: 'Churn', 0: 'No churn'},
                                            inplace=True)
    return predictions['Churn_prediction']


if __name__ == "__main__":
    df = load_data('C:/Users/mleop/OneDrive/Desktop/MSDS600/prepped_churn_data_final2.csv')
    predictions = make_predictions(df)
    print('predictions:')
    print(predictions)

In [41]:
%run predict_churn.py

Transformation Pipeline and Model Successfully Loaded


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Linear Discriminant Analysis,0.7955,0.8366,0.4794,0.6574,0.5545,0.4260,0.4349


predictions:
customerID
7590-VHVEG       Churn
5575-GNVDE    No churn
3668-QPYBK    No churn
7795-CFOCW    No churn
9237-HQITU       Churn
                ...   
6840-RESVB    No churn
2234-XADUH    No churn
4801-JZAZL    No churn
8361-LTMKD    No churn
3186-AJIEK    No churn
Name: Churn_prediction, Length: 7043, dtype: object


Summary: I first installed all the needed packages and imported the churn dataset. I think used automl to determine the best model which was LDA. I then loaded the model and saved it. Then using VSCode created a .py file to easily run the code on new data in the future. 